In [157]:
import os
from Bio import SeqIO
from Bio.Seq import Seq
from collections import Counter
from typing import List, Dict

START_CODON = "ATG"
STOP_CODONS = ["TAA", "TAG", "TGA"]
FASTA_DIRECTORY = "viruses/data"
file_prefixes = ["bacterial", "mamalian"]
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

In [158]:
class Virus:
    def __init__(self, fileName, name, sequence, orfs):
        self.fileName = fileName
        self.name = name
        self.sequence = sequence  # Original sequence
        self.orfs = orfs  # Sequence start-end
        self.protein_sequences = []  # List of protein sequences
        self.proteinsDicodones = []  # Stores dicodon frequencies
        self.codon_frequency = []  # Stores codon frequencies
        self.dicodon_frequency = []  # Stores dicodon frequencies
        self.amino_acid_frequencies = {}

    def calculate_frequencies(self):
        combined_sequence = ''.join(self.protein_sequences)
        self.amino_acid_frequencies = self.calculate_amino_acid_frequencies(combined_sequence)
        self.dicodon_frequency = self.calculate_dicodon_frequencies(combined_sequence)

    @staticmethod
    def calculate_amino_acid_frequencies(sequence: str) -> Dict[str, float]:
        amino_acid_count = Counter(sequence)
        total_amino_acids = sum(amino_acid_count.values())
        return {amino_acid: count / total_amino_acids for amino_acid, count in amino_acid_count.items()}

    @staticmethod
    def calculate_dicodon_frequencies(sequence: str) -> Dict[str, float]:
        dicodons = [sequence[i:i + 2] for i in range(len(sequence) - 1)]
        dicodon_count = Counter(dicodons)
        total_dicodons = sum(dicodon_count.values())
        return {dicodon: count / total_dicodons for dicodon, count in dicodon_count.items()}


In [159]:
def validate_sequence(seq):
    
    if START_CODON not in seq:
        return False, "No start codon found"
    
    if seq[-3:] not in STOP_CODONS:
        return False, "No valid stop codon found"
    
    if len(seq) % 3 != 0:
        return False, "Sequence length is not divisible by 3"
    
    if len(seq) < 100:
        return False, "Sequence length is less than 100 bp"
    
    return True, "Sequence is valid"

In [160]:
def find_orfs(sequence):
    orfs = []
    seq_len = len(sequence)
    
    for strand, nuc_seq in [(+1, sequence), (-1, sequence.reverse_complement())]:
        for frame in range(3):
            start_pos = None
            for i in range(frame, seq_len - 2, 3):
                codon = str(nuc_seq[i:i+3])
                if codon == START_CODON:
                    start_pos = i
                elif codon in STOP_CODONS and start_pos is not None:
                    stop_pos = i + 3
                    if (stop_pos - start_pos) >= 100:
                        orfs.append(str(nuc_seq[start_pos:stop_pos]))
                    start_pos = None
    return orfs

In [161]:
def process_fasta(file_path):
    all_orfs = []
    for record in SeqIO.parse(file_path, "fasta"):
        orfs = find_orfs(record.seq)
        all_orfs.extend(orfs)
    return all_orfs, record

In [162]:
def process_files(fasta_directory, file_prefixes):
    for prefix in file_prefixes:
        file_path = f"{fasta_directory}/{prefix}.fasta"
        orfs, record = process_fasta(file_path)
        virus = Virus(fileName=f"{prefix}.fasta", name=record.name, sequence=record.seq, orfs=orfs)
    return virus

In [163]:
def translate_to_protein(virus):
    protein_sequences = []
    proteinsDicodones = []

    for sequence  in virus.orfs:
        protein = Seq(sequence).translate()
        protein_sequences.append(str(protein))

        dicodons = [str(protein[i:i+2]) for i in range(len(protein)-1)]
        proteinsDicodones.append(dicodons[:-1])
    return protein_sequences, proteinsDicodones

In [164]:
# Surinkti duomenis ir start end poras
viruses = []
for i in range(1, 5):
    for prefix in file_prefixes:
        virus = process_files(FASTA_DIRECTORY, [f"{prefix}{i}"])
        viruses.append(virus)

In [165]:
# Konvertuoti sekas į baltymo kodonus ir dikodonus
for virus in viruses:
    virus.protein_sequences, virus.proteinsDicodones = translate_to_protein(virus)

In [166]:
def codon_dicodon_frequency(protein_codons, protein_dicodons):
    # Patikrinti, ar kodonai ir dikodonai nėra tušti
    if not protein_codons:
        raise ValueError("Protein codons list is empty.")
    if not protein_dicodons:
        raise ValueError("Protein dicodons list is empty.")

    # Inicializuoti žodynus kodonų ir dikodonų dažniams
    codon_frequency = {}
    dicodon_frequency = {}

    # Skaičiuoti kodonų dažnį
    for codon in protein_codons:
        codon_frequency[codon] = codon_frequency.get(codon, 0) + 1

    # Skaičiuoti dikodonų dažnį
    for dicodon in protein_dicodons:
        dicodon_frequency[dicodon] = dicodon_frequency.get(dicodon, 0) + 1

    # Normalizuoti kodonų dažnį pagal bendrą kodonų skaičių
    total_codons = len(protein_codons)
    codon_frequency = {codon: freq / total_codons for codon, freq in codon_frequency.items()}

    # Normalizuoti dikodonų dažnį pagal bendrą dikodonų skaičių
    total_dicodons = len(protein_dicodons)  # Dikodonų skaičius yra tikrasis įrašytų dikodonų kiekis
    dicodon_frequency = {dicodon: freq / total_dicodons for dicodon, freq in dicodon_frequency.items()}

    return codon_frequency, dicodon_frequency


In [167]:
def get_amino_acid_frequency(protein_seq):
    amino_acid_count = Counter(protein_seq)
    amino_acid_freq = {aa: amino_acid_count.get(aa, 0) for aa in amino_acids}
    return amino_acid_freq

def get_dipeptide_frequency(protein_seq):
    dipeptides = [protein_seq[i:i+2] for i in range(len(protein_seq)-1)]
    dipeptide_count = Counter(dipeptides)
    possible_dipeptides = [a+b for a in amino_acids for b in amino_acids]
    dipeptide_freq = {dipeptide: dipeptide_count.get(dipeptide, 0) for dipeptide in possible_dipeptides}
    return dipeptide_freq

In [168]:
# Rasti kodonų ir dikodonų dažnius
for virus in viruses:
    for i in range(len(virus.orfs)):
        codon_freq = get_amino_acid_frequency(virus.protein_sequences[i])
        # dicodon_freq = get_dipeptide_frequency(virus.proteinsCodones[i])
        virus.codon_frequency.append(codon_freq)
        # virus.dicodon_frequency.append(dicodon_freq)

In [169]:
import numpy as np

def calculate_distance_matrix(viruses, use_dicodons=False):

    num_viruses = len(viruses)
    dist_matrix = np.zeros((num_viruses, num_viruses))
    
    if use_dicodons:
        get_frequencies = lambda virus, all_keys: get_frequency_values(virus.dicodon_frequency, all_keys)
    else:
        get_frequencies = lambda virus, all_keys: get_frequency_values(virus.codon_frequency, all_keys)

    all_keys = get_all_keys(viruses, use_dicodons)
    
    for i in range(num_viruses):
        for j in range(i, num_viruses):
            freq_i = get_frequencies(viruses[i], all_keys)
            freq_j = get_frequencies(viruses[j], all_keys)
            distance = euclidean_distance(freq_i, freq_j)
            dist_matrix[i][j] = distance
            dist_matrix[j][i] = distance
    
    return dist_matrix

def get_all_keys(viruses, use_dicodons):
    all_keys = set()
    for virus in viruses:
        if use_dicodons:
            if isinstance(virus.dicodon_frequency, dict):
                all_keys.update(virus.dicodon_frequency.keys())
            elif isinstance(virus.dicodon_frequency, list):
          
                pass
        else:
            if isinstance(virus.codon_frequency, dict):
                all_keys.update(virus.codon_frequency.keys())
            elif isinstance(virus.codon_frequency, list):
              
                pass
    return all_keys

def get_frequency_values(freq_data, all_keys):
    if isinstance(freq_data, dict):

        return [freq_data.get(key, 0) for key in sorted(all_keys)]
    elif isinstance(freq_data, list):
       
        return freq_data
    else:
        raise TypeError("Frequency data must be either a list or a dictionary")

def euclidean_distance(freq_i, freq_j):
    return np.linalg.norm(np.array(freq_i) - np.array(freq_j))


In [170]:
for virus in viruses:
    virus.calculate_frequencies()
    print(f"Amino Acid Frequencies for {virus.name}: {virus.amino_acid_frequencies}")
    print(f"Dicodon Frequencies for {virus.name}: {virus.dicodon_frequency}")

Amino Acid Frequencies for Lactococcus_phage: {'M': 0.016710875331564987, 'V': 0.05755968169761273, 'A': 0.05013262599469496, 'K': 0.08063660477453581, 'G': 0.0389920424403183, 'N': 0.05305039787798409, 'P': 0.039257294429708225, 'E': 0.06445623342175066, 'L': 0.11114058355437666, 'Y': 0.042440318302387266, 'T': 0.058090185676392576, 'W': 0.011140583554376658, 'R': 0.03183023872679045, 'Q': 0.03183023872679045, 'S': 0.08992042440318303, 'D': 0.0389920424403183, 'I': 0.07374005305039788, 'F': 0.058090185676392576, '*': 0.016710875331564987, 'C': 0.01803713527851459, 'H': 0.017241379310344827}
Dicodon Frequencies for Lactococcus_phage: {'MV': 0.0007959671000265323, 'VA': 0.0029185460334306182, 'AK': 0.007959671000265322, 'KA': 0.004775802600159193, 'AG': 0.0018572565667285751, 'GN': 0.0029185460334306182, 'NP': 0.0018572565667285751, 'PE': 0.0021225789334040858, 'EL': 0.010082249933669409, 'LY': 0.007429026266914301, 'YN': 0.0029185460334306182, 'PT': 0.00344919076678164, 'TE': 0.0039798

In [171]:
import numpy as np
from typing import List, Dict
from collections import Counter
from scipy.spatial.distance import euclidean

def get_unique_frequencies(viruses: List[Virus], frequency_type: str) -> Dict[str, int]:
    """
    Get a unified set of unique amino acids or dicodons across all viruses.
    
    Parameters:
        viruses (List[Virus]): List of Virus objects.
        frequency_type (str): 'amino_acid' or 'dicodon' to specify which frequencies to use.
        
    Returns:
        Dict: A dictionary where keys are unique amino acids or dicodons, and values are their index positions.
    """
    unique_frequencies = set()  # Use a set to store unique amino acids/dicodons
    
    for virus in viruses:
        if frequency_type == 'amino_acid':
            unique_frequencies.update(virus.amino_acid_frequencies.keys())
        elif frequency_type == 'dicodon':
            unique_frequencies.update(virus.dicodon_frequency.keys())
        else:
            raise ValueError(f"Unknown frequency type: {frequency_type}")
    
    return {item: idx for idx, item in enumerate(sorted(unique_frequencies))}

def calculate_distance_matrix(viruses: List[Virus], frequency_type: str = 'amino_acid') -> np.ndarray:
    """
    Calculate a distance matrix based on either amino acid or dicodon frequencies.
    
    Parameters:
        viruses (List[Virus]): List of Virus objects.
        frequency_type (str): 'amino_acid' or 'dicodon' to specify which frequencies to use.
        
    Returns:
        np.ndarray: A 2D numpy array representing the distance matrix between viruses.
    """
    num_viruses = len(viruses)
    distance_matrix = np.zeros((num_viruses, num_viruses))  # Initialize a square matrix

    # Get the common set of unique amino acids or dicodons
    unique_frequencies = get_unique_frequencies(viruses, frequency_type)

    # For each virus, create a vector based on the common unique set
    for i in range(num_viruses):
        # Initialize frequency vector with zeros
        frequency_vector_i = np.zeros(len(unique_frequencies))
        
        # Get the corresponding frequency data for the virus
        if frequency_type == 'amino_acid':
            for amino_acid, freq in viruses[i].amino_acid_frequencies.items():
                frequency_vector_i[unique_frequencies[amino_acid]] = freq
        elif frequency_type == 'dicodon':
            for dicodon, freq in viruses[i].dicodon_frequency.items():
                frequency_vector_i[unique_frequencies[dicodon]] = freq

        # Calculate the distance for the current virus against all others
        for j in range(i, num_viruses):
            frequency_vector_j = np.zeros(len(unique_frequencies))
            if frequency_type == 'amino_acid':
                for amino_acid, freq in viruses[j].amino_acid_frequencies.items():
                    frequency_vector_j[unique_frequencies[amino_acid]] = freq
            elif frequency_type == 'dicodon':
                for dicodon, freq in viruses[j].dicodon_frequency.items():
                    frequency_vector_j[unique_frequencies[dicodon]] = freq

            # Calculate Euclidean distance between the two vectors
            distance = euclidean(frequency_vector_i, frequency_vector_j)
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance  # Symmetric matrix

    return distance_matrix

In [172]:
# Calculate amino acid frequencies
for virus in viruses:
    virus.calculate_frequencies()

# Create distance matrix based on amino acid frequencies
amino_acid_distance_matrix = calculate_distance_matrix(viruses, frequency_type='amino_acid')
print("Amino Acid Distance Matrix:")
print(amino_acid_distance_matrix)

# Create distance matrix based on dicodon frequencies
dicodon_distance_matrix = calculate_distance_matrix(viruses, frequency_type='dicodon')
print("Dicodon Distance Matrix:")
print(dicodon_distance_matrix)

Amino Acid Distance Matrix:
[[0.         0.07377612 0.06823452 0.09700324 0.03174921 0.06879066
  0.04332591 0.12954523]
 [0.07377612 0.         0.06454763 0.07893121 0.06980494 0.08627981
  0.09753478 0.10615695]
 [0.06823452 0.06454763 0.         0.04803482 0.05302559 0.0760194
  0.07656178 0.0786711 ]
 [0.09700324 0.07893121 0.04803482 0.         0.08344147 0.11116018
  0.10969463 0.04571285]
 [0.03174921 0.06980494 0.05302559 0.08344147 0.         0.05913812
  0.03816115 0.11466692]
 [0.06879066 0.08627981 0.0760194  0.11116018 0.05913812 0.
  0.06752855 0.13833751]
 [0.04332591 0.09753478 0.07656178 0.10969463 0.03816115 0.06752855
  0.         0.1384125 ]
 [0.12954523 0.10615695 0.0786711  0.04571285 0.11466692 0.13833751
  0.1384125  0.        ]]
Dicodon Distance Matrix:
[[0.         0.0385369  0.03256766 0.04085071 0.02442426 0.03407637
  0.03310544 0.0516902 ]
 [0.0385369  0.         0.03566884 0.03896923 0.03582678 0.04272295
  0.04587262 0.04625389]
 [0.03256766 0.03566884 0

In [175]:
import numpy as np

def write_phylip_format(viruses: List[Virus], distance_matrix: np.ndarray, file_name: str):

    num_viruses = len(viruses)

    # Check if the distance matrix size matches the number of viruses
    if distance_matrix.shape[0] != num_viruses or distance_matrix.shape[1] != num_viruses:
        raise ValueError("Distance matrix dimensions do not match the number of viruses")

    # Create a list to hold the PHYLIP formatted strings
    phylip_lines = []
    
    # First line contains the number of viruses and the number of sites (len of the frequency vector)
    phylip_lines.append(f"{num_viruses}")
    
    # For each virus, add its name followed by the distances to all other viruses
    for i in range(num_viruses):
        virus_name = viruses[i].name
        distances = [f"{distance_matrix[i][j]:8.4f}" for j in range(num_viruses)]  # Fixed width (8 characters) and 4 decimals
        
        # Add space after virus name (adjust width here, <15 means 15-character width)
        phylip_lines.append(f"{virus_name:<15}{' '.join(distances)}")

    # Write the lines to a file
    with open(file_name, 'w') as file:
        file.write("\n".join(phylip_lines))

# Example Usage
file_name = "codon viruses_distance_matrix.phy"
write_phylip_format(viruses, amino_acid_distance_matrix, file_name)
file_name = "dicodon viruses_distance_matrix.phy"
write_phylip_format(viruses, dicodon_distance_matrix, file_name)
